# Genetic Algorithms Basics

Genetic Algorithms borrow inspiration from biological inheritance processes to attempt to solve optimization problems by generating a "population" of possible solutions (chromosomes) from which the most "fit" are selected for mating and mutation. By iterating this process over generations we expect the population to slowly converge upon the most "fit" region of our problem space which would correspond to the solution to the optimization task.

## Problem Representation (mapping)

In MGSurvE we are trying to solve the question **"Given a heterogeneous environment and a limited number of traps, where should we place the devices?"**, so this is the optimization task we need to map into a genetic algorithm paradigm. To do this, the positions of the traps are encoded into the chromosomes of the populations in pairs of alleles as follows:

![](../../media/fitness.png)

where a chromosome (or individual) would contain the position of the full set of traps. Once we have set the position of our traps we can calculate our fitness metric ($\phi$ in this case) by computing the time it would take for a random walker to fall into a trap given the current setup (more on this in the [following section](#fitness)). 

## Optimization Cycle

### Fitness

Calculating the fitness function is the most computationally-intensive part of the optimization process, as it involves solving [Markov's Fundamental Matrix](https://en.wikipedia.org/wiki/Absorbing_Markov_chain) on each potential solution of our landscape.

![](../../media/fits.png)

For all the details of how we compute our base fitness function have a look at our ["Mathematical Formulation" document](../more/math.ipynb).

### Selection

![](../../media/selection.png)


### Crossover

![](../../media/crossover.png)

### Mutation

Finally, we iterate through our population selecting some individuals for a mutation process.

![](../../media/mutation.png)

<hr>

# More Information

* [Holland's original paper](https://link.springer.com/chapter/10.1007/978-1-4684-8941-5_21)
* [DEAP's documentation](https://deap.readthedocs.io/en/master/)
* [Fahmi Nurfikri's illustrated tutorial](https://towardsdatascience.com/an-illustrated-guide-to-genetic-algorithm-ec5615c9ebe)